In [1]:
# ^ ===== COMPLETE GUI SYSTEM WITH CAMERA INITIALIZATION =====
# Merged: GUI Application + Essential Variables + Camera Service + Camera Init

import tkinter as tk
from tkinter import ttk
import threading
from PIL import Image, ImageTk
import queue
import time
import numpy as np
import cv2
import logging

# Additional imports for detection algorithms
try:
    from scipy.signal import find_peaks
except ImportError:
    print("⚠️ scipy not available - using basic peak detection")
    def find_peaks(data, height=None):
        """Simple peak detection fallback"""
        if len(data) < 3:
            return [], {}
        peaks = []
        for i in range(1, len(data)-1):
            if data[i] > data[i-1] and data[i] > data[i+1]:
                if height is None or data[i] >= height:
                    peaks.append(i)
        return peaks, {}

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# ===== ESSENTIAL VARIABLES INITIALIZATION =====
# Initialize default values if not already defined

if 'BOX_SIZE' not in globals():
    BOX_SIZE = 100
if 'BINARY_THRESHOLD_MODE1' not in globals():
    BINARY_THRESHOLD_MODE1 = 200
if 'CALC_LENGTH' not in globals():
    CALC_LENGTH = 60
if 'fourier_peak_threshold' not in globals():
    fourier_peak_threshold = 5.0
if 'LED_FREQ' not in globals():
    LED_FREQ = 2.0
if 'FREQ_TOL' not in globals():
    FREQ_TOL = 1.0
if 'ROI_STABLE_FRAMES_MODE1' not in globals():
    ROI_STABLE_FRAMES_MODE1 = 3

# Mode 2 defaults - Yellow color HSV range
if 'HSV_LOWER_BOUND' not in globals():
    HSV_LOWER_BOUND = np.array([20, 100, 100])  # Yellow lower bound
if 'HSV_UPPER_BOUND' not in globals():
    HSV_UPPER_BOUND = np.array([30, 255, 255])  # Yellow upper bound
if 'FLICKER_BUFFER_LENGTH' not in globals():
    FLICKER_BUFFER_LENGTH = 10
if 'FLICKER_DIFF_THRESHOLD' not in globals():
    FLICKER_DIFF_THRESHOLD = 100
if 'STABLE_FRAMES_THRESHOLD_MODE2' not in globals():
    STABLE_FRAMES_THRESHOLD_MODE2 = 5

# Mode 3 defaults
if 'TRACKER_TIMEOUT_SECONDS' not in globals():
    TRACKER_TIMEOUT_SECONDS = 5.0
if 'MAX_TRACKING_FAILURES' not in globals():
    MAX_TRACKING_FAILURES = 20
if 'ALIGNMENT_ANGLE_THRESHOLD' not in globals():
    ALIGNMENT_ANGLE_THRESHOLD = 0.5
if 'ALIGNMENT_DEADBAND' not in globals():
    ALIGNMENT_DEADBAND = 1.0
if 'LED_CIRCLE_SIZE_FACTOR' not in globals():
    LED_CIRCLE_SIZE_FACTOR = 0.05
if 'DISTANCE_SAFETY_CHECK' not in globals():
    DISTANCE_SAFETY_CHECK = 0

# System defaults
if 'mode' not in globals():
    mode = 1
if 'USE_RTSP' not in globals():
    USE_RTSP = True
if 'HAS_REUCAMERA' not in globals():
    HAS_REUCAMERA = False
if 'RTSP_URL' not in globals():
    RTSP_URL = "rtsp://169.254.104.28:8554/0/unicast"

print("✅ Essential variables initialized")

# ===== CAMERA SERVICE CLASS =====

class RTSPCameraService:
    """RTSP camera service for GUI integration"""
    def __init__(self, rtsp_url, username=None, password=None):
        self.rtsp_url = rtsp_url
        self.username = username
        self.password = password
        self.cap = None
        self.is_connected = False
        
    def connect(self):
        """Connect to RTSP camera"""
        try:
            if self.username and self.password:
                url_parts = self.rtsp_url.split('://')
                if len(url_parts) == 2:
                    protocol, rest = url_parts
                    full_url = f"{protocol}://{self.username}:{self.password}@{rest}"
                else:
                    full_url = self.rtsp_url
            else:
                full_url = self.rtsp_url
            
            print(f"Attempting RTSP connection...")
            self.cap = cv2.VideoCapture(full_url)
            
            if self.cap.isOpened():
                ret, frame = self.cap.read()
                if ret and frame is not None:
                    self.is_connected = True
                    print("✅ RTSP camera connected")
                    return True
                else:
                    print("❌ RTSP camera opened but no frame")
                    self.disconnect()
                    return False
            else:
                print("❌ Failed to open RTSP camera")
                return False
        except Exception as e:
            print(f"❌ RTSP connection error: {e}")
            self.is_connected = False
            return False
    
    def get_frame(self):
        """Get a frame from camera"""
        if not self.is_connected or not self.cap:
            return None
        try:
            ret, frame = self.cap.read()
            return frame if ret and frame is not None else None
        except Exception as e:
            print(f"⚠️ Frame capture error: {e}")
            return None
    
    def disconnect(self):
        """Disconnect from camera"""
        if self.cap:
            self.cap.release()
        self.is_connected = False
        print("📷 RTSP camera disconnected")

def initialize_camera():
    """Initialize camera service"""
    global camera_service
    print("🔍 Initializing camera...")
    
    if USE_RTSP:
        camera_service = RTSPCameraService(
            rtsp_url=RTSP_URL,
            username="fgcam",
            password="admin"
        )
        if camera_service.connect():
            print("✅ Camera service ready")
        else:
            print("⚠️ RTSP camera not available")
            camera_service = None
    else:
        print("ℹ️ RTSP disabled")
        camera_service = None
    
    globals()['reucam'] = None
    globals()['is_recording'] = False

# ===== LED DETECTION GUI CLASS =====

class LEDDetectionGUI:
    def __init__(self, master):
        self.master = master
        self.master.title("LED Detection System - Real-Time Control")
        self.master.geometry("1260x810")
        
        self.is_running = False
        self.frame_queue = queue.Queue(maxsize=3)
        self.parameter_lock = threading.Lock()
        self.last_status_update = 0
        
        self.init_parameter_vars()
        self.create_widgets()
        
        self.processing_thread = None
        print("🔧 GUI initialized")
        
    def init_parameter_vars(self):
        """Initialize tkinter variables for all parameters"""
        def safe_get(var_name, default):
            return globals().get(var_name, default)
        
        # Mode 1 parameters
        self.box_size_var = tk.IntVar(value=safe_get('BOX_SIZE', 100))
        self.binary_threshold_var = tk.IntVar(value=safe_get('BINARY_THRESHOLD_MODE1', 200))
        self.calc_length_var = tk.IntVar(value=safe_get('CALC_LENGTH', 60))
        self.fourier_threshold_var = tk.DoubleVar(value=safe_get('fourier_peak_threshold', 5.0))
        self.led_freq_var = tk.DoubleVar(value=safe_get('LED_FREQ', 2.0))
        self.freq_tol_var = tk.DoubleVar(value=safe_get('FREQ_TOL', 1.0))
        self.roi_stable_frames_var = tk.IntVar(value=safe_get('ROI_STABLE_FRAMES_MODE1', 3))
        
        # Mode 2 parameters
        hsv_lower = safe_get('HSV_LOWER_BOUND', np.array([0, 50, 50]))
        hsv_upper = safe_get('HSV_UPPER_BOUND', np.array([179, 255, 255]))
        self.hsv_lower_h_var = tk.IntVar(value=hsv_lower[0])
        self.hsv_lower_s_var = tk.IntVar(value=hsv_lower[1])
        self.hsv_lower_v_var = tk.IntVar(value=hsv_lower[2])
        self.hsv_upper_h_var = tk.IntVar(value=hsv_upper[0])
        self.hsv_upper_s_var = tk.IntVar(value=hsv_upper[1])
        self.hsv_upper_v_var = tk.IntVar(value=hsv_upper[2])
        self.flicker_buffer_var = tk.IntVar(value=safe_get('FLICKER_BUFFER_LENGTH', 10))
        self.flicker_diff_var = tk.IntVar(value=safe_get('FLICKER_DIFF_THRESHOLD', 100))
        self.stable_frames_mode2_var = tk.IntVar(value=safe_get('STABLE_FRAMES_THRESHOLD_MODE2', 5))
        
        # Mode 3 parameters
        self.tracker_timeout_var = tk.DoubleVar(value=safe_get('TRACKER_TIMEOUT_SECONDS', 5.0))
        self.max_tracking_failures_var = tk.IntVar(value=safe_get('MAX_TRACKING_FAILURES', 20))
        self.alignment_angle_threshold_var = tk.DoubleVar(value=safe_get('ALIGNMENT_ANGLE_THRESHOLD', 0.5))
        self.alignment_deadband_var = tk.DoubleVar(value=safe_get('ALIGNMENT_DEADBAND', 1.0))
        self.led_circle_size_var = tk.DoubleVar(value=safe_get('LED_CIRCLE_SIZE_FACTOR', 0.05))
        self.distance_safety_var = tk.DoubleVar(value=safe_get('DISTANCE_SAFETY_CHECK', 0))
        
        # System status
        current_mode = safe_get('mode', 1)
        self.current_mode_var = tk.StringVar(value=f"Mode {current_mode}")
        self.fps_var = tk.StringVar(value="0.0")
        self.distance_var = tk.StringVar(value="N/A")
        self.alignment_var = tk.StringVar(value="N/A")
        self.camera_status_var = tk.StringVar(value="Disconnected")
        
    def create_widgets(self):
        """Create main GUI layout"""
        main_frame = ttk.Frame(self.master)
        main_frame.pack(fill=tk.BOTH, expand=True, padx=8, pady=8)
        
        # Top: Mode selection buttons
        self.create_mode_buttons(main_frame)
        
        # Content frame
        content_frame = ttk.Frame(main_frame)
        content_frame.pack(fill=tk.BOTH, expand=True, pady=(8, 0))
        
        # Left: Video display
        left_frame = ttk.Frame(content_frame)
        left_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=(0, 8))
        self.create_video_panel(left_frame)
        
        # Right: Legend and parameters
        right_frame = ttk.Frame(content_frame, width=360)
        right_frame.pack(side=tk.RIGHT, fill=tk.BOTH)
        
        self.create_color_legend(right_frame)
        self.create_parameter_panels(right_frame)
        
        # Safety notice
        safety_frame = ttk.Frame(right_frame)
        safety_frame.pack(fill=tk.X, pady=(8, 0))
        ttk.Label(safety_frame, 
                 text="⚠️ LIVE STREAM SAFETY:\nSome parameters require stopping\ndetection before changing",
                 font=("Arial", 8), foreground="red", justify=tk.CENTER).pack()
        
    def create_video_panel(self, parent):
        """Create video display area"""
        video_frame = ttk.LabelFrame(parent, text="Video Feed", padding=8)
        video_frame.pack(fill=tk.BOTH, expand=True, pady=(0, 8))
        
        self.video_canvas = tk.Canvas(video_frame, bg='black', width=576, height=432)
        self.video_canvas.pack(expand=True)
        
        # Control buttons
        control_frame = ttk.Frame(video_frame)
        control_frame.pack(fill=tk.X, pady=(8, 8))
        
        self.start_btn = ttk.Button(control_frame, text="▶ Start Detection", 
                                    command=self.start_detection, width=18)
        self.start_btn.pack(side=tk.LEFT, padx=(0, 5))
        
        self.stop_btn = ttk.Button(control_frame, text="⏹ Stop Detection", 
                                   command=self.stop_detection, width=18, state=tk.DISABLED)
        self.stop_btn.pack(side=tk.LEFT, padx=(0, 5))
        
        self.record_btn = ttk.Button(control_frame, text="⏺ Start Recording", 
                                     command=self.toggle_recording, width=18)
        self.record_btn.pack(side=tk.LEFT)
        
        # Status display
        status_frame = ttk.Frame(video_frame)
        status_frame.pack(fill=tk.X, pady=(8, 0))
        
        ttk.Label(status_frame, text="Mode:").grid(row=0, column=0, sticky=tk.W)
        ttk.Label(status_frame, textvariable=self.current_mode_var, font=("Arial", 10, "bold")).grid(row=0, column=1, sticky=tk.W, padx=(5, 20))
        ttk.Label(status_frame, text="Camera:").grid(row=0, column=2, sticky=tk.W)
        ttk.Label(status_frame, textvariable=self.camera_status_var).grid(row=0, column=3, sticky=tk.W, padx=(5, 20))
        ttk.Label(status_frame, text="FPS:").grid(row=1, column=0, sticky=tk.W)
        ttk.Label(status_frame, textvariable=self.fps_var).grid(row=1, column=1, sticky=tk.W, padx=(5, 20))
        ttk.Label(status_frame, text="Distance:").grid(row=1, column=2, sticky=tk.W)
        ttk.Label(status_frame, textvariable=self.distance_var).grid(row=1, column=3, sticky=tk.W, padx=(5, 0))
    
    def create_mode_buttons(self, parent):
        """Create mode selection buttons at top"""
        mode_frame = ttk.LabelFrame(parent, text="Detection Modes", padding=8)
        mode_frame.pack(fill=tk.X, pady=(0, 8))
        
        button_container = ttk.Frame(mode_frame)
        button_container.pack()
        
        self.mode1_btn = ttk.Button(button_container, text="Mode 1: FFT Frequency", 
                                    command=self.force_mode1, width=22)
        self.mode1_btn.grid(row=0, column=0, padx=5, pady=2)
        
        self.mode2_btn = ttk.Button(button_container, text="Mode 2: Yellow + Flicker", 
                                    command=self.force_mode2, width=22)
        self.mode2_btn.grid(row=0, column=1, padx=5, pady=2)
        
        self.mode3_btn = ttk.Button(button_container, text="Mode 3: Object Tracking", 
                                    command=self.force_mode3, width=22)
        self.mode3_btn.grid(row=0, column=2, padx=5, pady=2)
    
    def create_color_legend(self, parent):
        """Create color legend panel"""
        legend_frame = ttk.LabelFrame(parent, text="Color Legend", padding=8)
        legend_frame.pack(fill=tk.X, pady=(0, 8))
        
        self.legend_text = tk.Text(legend_frame, height=5, width=40, 
                                   font=("Arial", 9), wrap=tk.WORD,
                                   bg="white", relief="sunken", borderwidth=1)
        self.legend_text.pack(fill=tk.BOTH, expand=True)
        
        # Configure color tags
        self.legend_text.tag_configure("yellow", foreground="#FFD700", font=("Arial", 9, "bold"))
        self.legend_text.tag_configure("orange", foreground="#FFA500", font=("Arial", 9, "bold"))
        self.legend_text.tag_configure("blue", foreground="#0066FF", font=("Arial", 9, "bold"))
        self.legend_text.tag_configure("gray", foreground="#666666", font=("Arial", 9, "bold"))
        self.legend_text.tag_configure("green", foreground="#00FF00", font=("Arial", 9, "bold"))
        self.legend_text.tag_configure("magenta", foreground="#FF00FF", font=("Arial", 9, "bold"))
        self.legend_text.tag_configure("title", font=("Arial", 9, "bold"))
        
        self.update_legend_display(1)
    
    def update_legend_display(self, mode):
        """Update legend based on current mode"""
        self.legend_text.config(state=tk.NORMAL)
        self.legend_text.delete(1.0, tk.END)
        
        if mode == 1:
            self.legend_text.insert(tk.END, "Mode 1: FFT Frequency Detection\n", "title")
            self.legend_text.insert(tk.END, "• ")
            self.legend_text.insert(tk.END, "Green", "green")
            self.legend_text.insert(tk.END, " = Target freq (~2Hz)\n")
            self.legend_text.insert(tk.END, "• ")
            self.legend_text.insert(tk.END, "Yellow", "yellow")
            self.legend_text.insert(tk.END, " = Other frequency detected\n")
            self.legend_text.insert(tk.END, "• ")
            self.legend_text.insert(tk.END, "Blue", "blue")
            self.legend_text.insert(tk.END, " = Bright, no frequency\n")
            self.legend_text.insert(tk.END, "• Shows Hz value on each box")
        elif mode == 2:
            self.legend_text.insert(tk.END, "Mode 2: Yellow HSV + Flicker\n", "title")
            self.legend_text.insert(tk.END, "• RED = Strong flicker (LED!)\n")
            self.legend_text.insert(tk.END, "• ")
            self.legend_text.insert(tk.END, "Yellow", "yellow")
            self.legend_text.insert(tk.END, " = Weak flicker\n")
            self.legend_text.insert(tk.END, "• ")
            self.legend_text.insert(tk.END, "Blue", "blue")
            self.legend_text.insert(tk.END, " = Yellow color, no flicker\n")
            self.legend_text.insert(tk.END, "• Grid shows intensity difference (Δ)")
        elif mode == 3:
            self.legend_text.insert(tk.END, "Mode 3: Object Detection\n", "title")
            self.legend_text.insert(tk.END, "• ")
            self.legend_text.insert(tk.END, "Green", "green")
            self.legend_text.insert(tk.END, " = 1st largest object\n")
            self.legend_text.insert(tk.END, "• ")
            self.legend_text.insert(tk.END, "Yellow", "yellow")
            self.legend_text.insert(tk.END, " = 2nd largest object\n")
            self.legend_text.insert(tk.END, "• ")
            self.legend_text.insert(tk.END, "Magenta", "magenta")
            self.legend_text.insert(tk.END, " = 3rd largest object\n")
            self.legend_text.insert(tk.END, "• Visual analysis only (no tracking)")
        
        self.legend_text.config(state=tk.DISABLED)
    
    def create_parameter_panels(self, parent):
        """Create parameter control panels"""
        notebook = ttk.Notebook(parent)
        notebook.pack(fill=tk.BOTH, expand=True)
        
        # Mode 1 tab
        mode1_frame = ttk.Frame(notebook)
        notebook.add(mode1_frame, text="Mode 1: Frequency Detection")
        self.create_mode1_controls(mode1_frame)
        
        # Mode 2 tab
        mode2_frame = ttk.Frame(notebook)
        notebook.add(mode2_frame, text="Mode 2: Color Filtering")
        self.create_mode2_controls(mode2_frame)
        
        # Mode 3 tab
        mode3_frame = ttk.Frame(notebook)
        notebook.add(mode3_frame, text="Mode 3: Object Tracking")
        self.create_mode3_controls(mode3_frame)
    
    def create_mode1_controls(self, parent):
        """Create Mode 1 parameter controls"""
        scroll_frame = self.create_scrollable_frame(parent)
        
        row = 0
        ttk.Label(scroll_frame, text="Detection Parameters", font=("Arial", 10, "bold")).grid(row=row, column=0, columnspan=5, sticky=tk.W, pady=(0, 8))
        row += 1
        
        self.create_slider_with_buttons(scroll_frame, "🚫 Box Size (pixels)", self.box_size_var, 50, 200, row)
        row += 1
        self.create_slider_with_buttons(scroll_frame, "Binary Threshold", self.binary_threshold_var, 100, 255, row)
        row += 1
        self.create_slider_with_buttons(scroll_frame, "🚫 Calculation Length (frames)", self.calc_length_var, 20, 120, row)
        row += 1
        
        ttk.Label(scroll_frame, text="FFT Parameters", font=("Arial", 10, "bold")).grid(row=row, column=0, columnspan=5, sticky=tk.W, pady=(15, 8))
        row += 1
        
        self.create_slider_with_buttons(scroll_frame, "Fourier Threshold", self.fourier_threshold_var, 1.0, 20.0, row, resolution=0.1)
        row += 1
        self.create_slider_with_buttons(scroll_frame, "LED Frequency (Hz)", self.led_freq_var, 0.5, 10.0, row, resolution=0.1)
        row += 1
        self.create_slider_with_buttons(scroll_frame, "Frequency Tolerance", self.freq_tol_var, 0.1, 3.0, row, resolution=0.1)
        row += 1
        self.create_slider_with_buttons(scroll_frame, "ROI Stable Frames", self.roi_stable_frames_var, 1, 10, row)
    
    def create_mode2_controls(self, parent):
        """Create Mode 2 parameter controls"""
        scroll_frame = self.create_scrollable_frame(parent)
        
        row = 0
        ttk.Label(scroll_frame, text="HSV Color Range", font=("Arial", 10, "bold")).grid(row=row, column=0, columnspan=5, sticky=tk.W, pady=(0, 8))
        row += 1
        
        ttk.Label(scroll_frame, text="Lower Bounds:", font=("Arial", 9, "bold")).grid(row=row, column=0, columnspan=5, sticky=tk.W)
        row += 1
        self.create_slider_with_buttons(scroll_frame, "Hue Lower", self.hsv_lower_h_var, 0, 179, row)
        row += 1
        self.create_slider_with_buttons(scroll_frame, "Saturation Lower", self.hsv_lower_s_var, 0, 255, row)
        row += 1
        self.create_slider_with_buttons(scroll_frame, "Value Lower", self.hsv_lower_v_var, 0, 255, row)
        row += 1
        
        ttk.Label(scroll_frame, text="Upper Bounds:", font=("Arial", 9, "bold")).grid(row=row, column=0, columnspan=5, sticky=tk.W, pady=(8, 0))
        row += 1
        self.create_slider_with_buttons(scroll_frame, "Hue Upper", self.hsv_upper_h_var, 0, 179, row)
        row += 1
        self.create_slider_with_buttons(scroll_frame, "Saturation Upper", self.hsv_upper_s_var, 0, 255, row)
        row += 1
        self.create_slider_with_buttons(scroll_frame, "Value Upper", self.hsv_upper_v_var, 0, 255, row)
        row += 1
        
        ttk.Label(scroll_frame, text="Flicker Analysis", font=("Arial", 10, "bold")).grid(row=row, column=0, columnspan=5, sticky=tk.W, pady=(15, 8))
        row += 1
        self.create_slider_with_buttons(scroll_frame, "🚫 Flicker Buffer Length", self.flicker_buffer_var, 5, 30, row)
        row += 1
        self.create_slider_with_buttons(scroll_frame, "Flicker Diff Threshold", self.flicker_diff_var, 50, 255, row)
        row += 1
        self.create_slider_with_buttons(scroll_frame, "Stable Frames Threshold", self.stable_frames_mode2_var, 1, 20, row)
    
    def create_mode3_controls(self, parent):
        """Create Mode 3 parameter controls"""
        scroll_frame = self.create_scrollable_frame(parent)
        
        row = 0
        ttk.Label(scroll_frame, text="Tracking Parameters", font=("Arial", 10, "bold")).grid(row=row, column=0, columnspan=5, sticky=tk.W, pady=(0, 8))
        row += 1
        
        self.create_slider_with_buttons(scroll_frame, "Tracker Timeout (s)", self.tracker_timeout_var, 1.0, 20.0, row, resolution=0.1)
        row += 1
        self.create_slider_with_buttons(scroll_frame, "Max Tracking Failures", self.max_tracking_failures_var, 10, 100, row)
        row += 1
        
        ttk.Label(scroll_frame, text="Alignment Parameters", font=("Arial", 10, "bold")).grid(row=row, column=0, columnspan=5, sticky=tk.W, pady=(15, 8))
        row += 1
        
        self.create_slider_with_buttons(scroll_frame, "Angle Threshold (°)", self.alignment_angle_threshold_var, 0.1, 2.0, row, resolution=0.1)
        row += 1
        self.create_slider_with_buttons(scroll_frame, "Alignment Deadband (°)", self.alignment_deadband_var, 0.1, 3.0, row, resolution=0.1)
        row += 1
        
        ttk.Label(scroll_frame, text="Distance Calculation", font=("Arial", 10, "bold")).grid(row=row, column=0, columnspan=5, sticky=tk.W, pady=(15, 8))
        row += 1
        
        self.create_slider_with_buttons(scroll_frame, "LED Circle Size Factor", self.led_circle_size_var, 0.01, 0.2, row, resolution=0.001)
        row += 1
        self.create_slider_with_buttons(scroll_frame, "Distance Safety Check", self.distance_safety_var, -5.0, 5.0, row, resolution=0.1)
    
    def create_scrollable_frame(self, parent):
        """Create a scrollable frame"""
        canvas = tk.Canvas(parent)
        scrollbar = ttk.Scrollbar(parent, orient="vertical", command=canvas.yview)
        scrollable_frame = ttk.Frame(canvas)
        
        scrollable_frame.bind("<Configure>", lambda e: canvas.configure(scrollregion=canvas.bbox("all")))
        canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
        canvas.configure(yscrollcommand=scrollbar.set)
        
        canvas.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")
        
        return scrollable_frame
    
    def create_slider_with_buttons(self, parent, label, var, min_val, max_val, row, resolution=1):
        """Create slider control with +/- buttons"""
        ttk.Label(parent, text=label, font=("Arial", 8)).grid(row=row, column=0, sticky=tk.W, padx=(0, 5))
        
        ttk.Button(parent, text="-", width=2,
                  command=lambda: self.adjust_parameter(var, -resolution, min_val, max_val)).grid(row=row, column=1, padx=(0, 2))
        
        slider = ttk.Scale(parent, from_=min_val, to=max_val, variable=var, orient=tk.HORIZONTAL, length=140)
        slider.configure(command=lambda v: self.update_parameter(var, v))
        slider.grid(row=row, column=2, sticky=tk.W, padx=2)
        
        ttk.Button(parent, text="+", width=2,
                  command=lambda: self.adjust_parameter(var, resolution, min_val, max_val)).grid(row=row, column=3, padx=(2, 5))
        
        try:
            current_value = var.get()
            value_text = f"{current_value:.3f}" if resolution < 1 else f"{int(current_value)}"
        except Exception:
            value_text = "0"
            
        value_label = ttk.Label(parent, text=value_text, font=("Arial", 9, "bold"), width=8, 
                               foreground="black", background="lightgray", relief="sunken")
        value_label.grid(row=row, column=4, sticky=tk.W, padx=(5, 0))
        
        var.value_label = value_label
        var.resolution = resolution
        var.min_val = min_val
        var.max_val = max_val
    
    def adjust_parameter(self, var, adjustment, min_val, max_val):
        """Adjust parameter by increment/decrement"""
        try:
            current_value = var.get()
            new_value = max(min_val, min(max_val, current_value + adjustment))
            var.set(new_value)
            
            if hasattr(var, 'value_label') and hasattr(var, 'resolution'):
                if var.resolution >= 1:
                    var.value_label.config(text=f"{int(new_value)}")
                else:
                    var.value_label.config(text=f"{new_value:.3f}")
            
            self.update_parameter(var, new_value)
        except Exception as e:
            print(f"⚠️ Parameter adjustment error: {e}")
    
    def update_parameter(self, var, value):
        """Update parameter with safety checks"""
        try:
            if hasattr(var, 'resolution') and hasattr(var, 'value_label'):
                try:
                    if var.resolution >= 1:
                        var.value_label.config(text=f"{int(float(value))}")
                    else:
                        var.value_label.config(text=f"{float(value):.3f}")
                except Exception:
                    pass
        except Exception:
            pass
        
        dangerous_params = [self.calc_length_var, self.box_size_var, self.flicker_buffer_var]
        
        if self.is_running and var in dangerous_params:
            param_name = ""
            if var == self.calc_length_var:
                param_name = "Calculation Length"
            elif var == self.box_size_var:
                param_name = "Box Size"
            elif var == self.flicker_buffer_var:
                param_name = "Flicker Buffer Length"
            
            try:
                def show_warning():
                    self.show_parameter_warning(param_name)
                self.master.after_idle(show_warning)
            except Exception:
                pass
            return
        
        try:
            self.apply_safe_parameters()
        except Exception as e:
            print(f"⚠️ Parameter update error: {e}")
    
    def apply_safe_parameters(self):
        """Apply parameters safely"""
        if not self.is_running:
            try:
                with self.parameter_lock:
                    self.apply_parameter_changes()
            except Exception as e:
                print(f"⚠️ Parameter application error: {e}")
    
    def apply_parameter_changes(self):
        """Apply GUI parameters to global variables"""
        if self.is_running:
            return
            
        global BOX_SIZE, BINARY_THRESHOLD_MODE1, CALC_LENGTH, fourier_peak_threshold
        global LED_FREQ, FREQ_TOL, ROI_STABLE_FRAMES_MODE1
        global HSV_LOWER_BOUND, HSV_UPPER_BOUND, FLICKER_BUFFER_LENGTH
        global FLICKER_DIFF_THRESHOLD, STABLE_FRAMES_THRESHOLD_MODE2
        global TRACKER_TIMEOUT_SECONDS, MAX_TRACKING_FAILURES
        global ALIGNMENT_ANGLE_THRESHOLD, ALIGNMENT_DEADBAND
        global LED_CIRCLE_SIZE_FACTOR, DISTANCE_SAFETY_CHECK
        
        try:
            BOX_SIZE = max(10, min(500, self.box_size_var.get()))
            BINARY_THRESHOLD_MODE1 = max(50, min(255, self.binary_threshold_var.get()))
            CALC_LENGTH = max(10, min(300, self.calc_length_var.get()))
            fourier_peak_threshold = max(0.1, min(50.0, self.fourier_threshold_var.get()))
            LED_FREQ = max(0.1, min(20.0, self.led_freq_var.get()))
            FREQ_TOL = max(0.1, min(10.0, self.freq_tol_var.get()))
            ROI_STABLE_FRAMES_MODE1 = max(1, min(20, self.roi_stable_frames_var.get()))
            
            h_lower = max(0, min(179, self.hsv_lower_h_var.get()))
            s_lower = max(0, min(255, self.hsv_lower_s_var.get()))
            v_lower = max(0, min(255, self.hsv_lower_v_var.get()))
            h_upper = max(h_lower, min(179, self.hsv_upper_h_var.get()))
            s_upper = max(s_lower, min(255, self.hsv_upper_s_var.get()))
            v_upper = max(v_lower, min(255, self.hsv_upper_v_var.get()))
            
            HSV_LOWER_BOUND = np.array([h_lower, s_lower, v_lower])
            HSV_UPPER_BOUND = np.array([h_upper, s_upper, v_upper])
            
            FLICKER_BUFFER_LENGTH = max(5, min(50, self.flicker_buffer_var.get()))
            FLICKER_DIFF_THRESHOLD = max(10, min(500, self.flicker_diff_var.get()))
            STABLE_FRAMES_THRESHOLD_MODE2 = max(1, min(30, self.stable_frames_mode2_var.get()))
            
            TRACKER_TIMEOUT_SECONDS = max(1.0, min(30.0, self.tracker_timeout_var.get()))
            MAX_TRACKING_FAILURES = max(5, min(200, self.max_tracking_failures_var.get()))
            ALIGNMENT_ANGLE_THRESHOLD = max(0.1, min(10.0, self.alignment_angle_threshold_var.get()))
            ALIGNMENT_DEADBAND = max(0.1, min(10.0, self.alignment_deadband_var.get()))
            LED_CIRCLE_SIZE_FACTOR = max(0.001, min(1.0, self.led_circle_size_var.get()))
            DISTANCE_SAFETY_CHECK = max(-10.0, min(10.0, self.distance_safety_var.get()))
        except Exception as e:
            print(f"⚠️ Parameter bounds checking error: {e}")
    
    def get_live_parameter(self, param_name):
        """Get parameter values during live stream"""
        try:
            if param_name == 'LED_FREQ':
                return max(0.1, min(20.0, self.led_freq_var.get()))
            elif param_name == 'FREQ_TOL':
                return max(0.1, min(10.0, self.freq_tol_var.get()))
            elif param_name == 'BINARY_THRESHOLD_MODE1':
                return max(50, min(255, self.binary_threshold_var.get()))
            elif param_name == 'HSV_LOWER_BOUND':
                return np.array([
                    max(0, min(179, self.hsv_lower_h_var.get())),
                    max(0, min(255, self.hsv_lower_s_var.get())),
                    max(0, min(255, self.hsv_lower_v_var.get()))
                ])
            elif param_name == 'HSV_UPPER_BOUND':
                return np.array([
                    max(0, min(179, self.hsv_upper_h_var.get())),
                    max(0, min(255, self.hsv_upper_s_var.get())),
                    max(0, min(255, self.hsv_upper_v_var.get()))
                ])
            elif param_name == 'FLICKER_DIFF_THRESHOLD':
                return max(10, min(500, self.flicker_diff_var.get()))
            elif param_name == 'MAX_TRACKING_FAILURES':
                return max(5, min(200, self.max_tracking_failures_var.get()))
            else:
                return globals().get(param_name, 0)
        except Exception:
            return globals().get(param_name, 0)
    
    def show_parameter_warning(self, param_name):
        """Show warning for dangerous parameter changes"""
        import tkinter.messagebox as messagebox
        messagebox.showwarning(
            "Parameter Warning",
            f"{param_name} cannot be changed during live detection.\n\n"
            f"Stop detection first, then adjust this parameter."
        )
    
    def start_detection(self):
        """Start detection"""
        if not self.is_running:
            self.is_running = True
            self.start_btn.config(state=tk.DISABLED)
            self.stop_btn.config(state=tk.NORMAL)
            
            self.last_status_update = 0
            
            global detection_engine
            try:
                if 'detection_engine' not in globals() or detection_engine is None:
                    from types import SimpleNamespace
                    detection_engine = SimpleNamespace()
                    detection_engine.frame_count = 0
                    detection_engine.fourier_buff = None
                    detection_engine.freq_counter = None
                    detection_engine.HE = 0
                    detection_engine.WI = 0
                    detection_engine.flicker_buff = None
                    detection_engine.flicker_stable_count = 0
            except Exception as e:
                print(f"Engine init error: {e}")
            
            self.processing_thread = threading.Thread(target=self.run_detection_loop, daemon=True)
            self.processing_thread.start()
            
            self.update_display()
            print("✅ Detection started")
    
    def stop_detection(self):
        """Stop detection"""
        if self.is_running:
            self.is_running = False
            self.start_btn.config(state=tk.NORMAL)
            self.stop_btn.config(state=tk.DISABLED)
            
            if hasattr(self, 'processing_thread') and self.processing_thread and self.processing_thread.is_alive():
                try:
                    self.processing_thread.join(timeout=2.0)
                except Exception:
                    pass
            
            print("⏹️ Detection stopped")
    
    def force_mode1(self):
        """Switch to Mode 1"""
        global mode
        mode = 1
        
        if 'detection_engine' in globals():
            detection_engine.fourier_buff = None
            detection_engine.freq_counter = None
            detection_engine.HE = 0
            detection_engine.WI = 0
            print("🔄 Mode 1: Grid analysis mode")
        
        self.update_legend_display(1)
        print("📊 Mode 1: Grid pixel analysis")
    
    def force_mode2(self):
        """Switch to Mode 2"""
        global mode
        mode = 2
        
        if 'detection_engine' in globals():
            detection_engine.flicker_buff = None
            detection_engine.flicker_stable_count = 0
            print("🔄 Mode 2: HSV filtering mode")
        
        self.update_legend_display(2)
        print("🎨 Mode 2: HSV color filtering")
    
    def force_mode3(self):
        """Switch to Mode 3"""
        global mode
        mode = 3
        
        if 'detection_engine' in globals():
            detection_engine.tracker = None
            detection_engine.bbox_mode3 = None
            print("🔄 Mode 3: Object visualization mode")
        
        self.update_legend_display(3)
        print("👁️ Mode 3: Object detection")
    
    def toggle_recording(self):
        """Toggle video recording"""
        is_recording = globals().get('is_recording', False)
        
        if is_recording:
            if 'stop_recording' in globals():
                globals()['stop_recording']()
            self.record_btn.config(text="Start Recording")
            globals()['is_recording'] = False
        else:
            if 'start_recording' in globals():
                globals()['start_recording']()
            self.record_btn.config(text="Stop Recording")
            globals()['is_recording'] = True
    
    def run_detection_loop(self):
        """Main detection processing loop"""
        global camera_service
        
        # Initialize camera on first run if needed
        if USE_RTSP and camera_service is None:
            print("🔌 Connecting to RTSP camera...")
            camera_service = RTSPCameraService(
                rtsp_url=RTSP_URL,
                username="fgcam",
                password="admin"
            )
            if camera_service.connect():
                print("✅ RTSP camera connected successfully")
                self.master.after(0, lambda: self.camera_status_var.set("RTSP Connected"))
            else:
                print("⚠️ RTSP camera connection failed")
                camera_service = None
                self.master.after(0, lambda: self.camera_status_var.set("RTSP Failed"))
        
        reucam = globals().get('reucam', None)
        
        frame_count = 0
        fps_start = time.time()
        last_gui_update = time.time()
        gui_update_interval = 0.5
        
        while self.is_running:
            try:
                frame = None
                camera_status = "No Camera"
                
                if USE_RTSP and camera_service:
                    try:
                        frame = camera_service.get_frame()
                        if frame is not None:
                            camera_status = "RTSP Connected"
                        else:
                            camera_status = "RTSP Failed"
                    except Exception:
                        camera_status = "RTSP Error"
                
                if frame is None and HAS_REUCAMERA and reucam:
                    try:
                        yuv_data = reucam.read()
                        if yuv_data:
                            frame = self.process_yuv_data(yuv_data)
                            camera_status = "ReuCamera Connected"
                    except Exception:
                        camera_status = "ReuCamera Failed"
                
                # Generate test frame if no camera available
                if frame is None:
                    frame = self.generate_test_frame()
                    camera_status = "Test Mode (No Camera)"
                
                if frame is not None:
                    processed_frame = self.process_frame_for_gui(frame.copy())
                    
                    try:
                        if not self.frame_queue.full():
                            self.frame_queue.put_nowait(processed_frame)
                        else:
                            try:
                                self.frame_queue.get_nowait()
                                self.frame_queue.put_nowait(processed_frame)
                            except queue.Empty:
                                pass
                    except queue.Full:
                        pass
                    
                    frame_count += 1
                    current_time = time.time()
                    
                    if current_time - last_gui_update >= gui_update_interval:
                        last_gui_update = current_time
                        
                        if current_time - fps_start >= 1.0:
                            fps = frame_count / (current_time - fps_start)
                            frame_count = 0
                            fps_start = current_time
                            
                            try:
                                self._safe_gui_update('fps', f"{fps:.1f}")
                            except Exception:
                                pass
                        
                        try:
                            self._safe_gui_update('mode', f"Mode {mode}")
                            self._safe_gui_update('camera', camera_status)
                        except Exception:
                            pass
                    
                    time.sleep(0.033)
                else:
                    try:
                        self._safe_gui_update('camera', camera_status)
                    except Exception:
                        pass
                    time.sleep(0.1)
            except Exception as e:
                logging.error(f"Detection loop error: {e}")
                try:
                    self._safe_gui_update('camera', "Error")
                except Exception:
                    pass
                time.sleep(0.1)
    
    def _safe_gui_update(self, update_type, value):
        """Safely update GUI variables"""
        try:
            if update_type == 'fps' and hasattr(self, 'fps_var'):
                self.fps_var.set(value)
            elif update_type == 'mode' and hasattr(self, 'current_mode_var'):
                self.current_mode_var.set(value)
            elif update_type == 'camera' and hasattr(self, 'camera_status_var'):
                self.camera_status_var.set(value)
            elif update_type == 'distance' and hasattr(self, 'distance_var'):
                self.distance_var.set(value)
            elif update_type == 'alignment' and hasattr(self, 'alignment_var'):
                self.alignment_var.set(value)
        except Exception as e:
            logging.warning(f"GUI update failed for {update_type}: {e}")
    
    def process_yuv_data(self, yuv_data):
        """Process YUV data to RGB frame"""
        return np.zeros((480, 640, 3), dtype=np.uint8)
    
    def generate_test_frame(self):
        """Generate test frame with simulated 2Hz flickering LEDs"""
        if not hasattr(self, 'test_frame_count'):
            self.test_frame_count = 0
        
        self.test_frame_count += 1
        
        # Create background with some noise
        frame = np.random.randint(40, 80, (480, 640, 3), dtype=np.uint8)
        
        # Add 3 simulated yellow LEDs with 2Hz flicker (at 30fps, period = 15 frames)
        led_positions = [(150, 200), (320, 240), (500, 280)]
        
        for x, y in led_positions:
            # 2Hz sine wave: flickers twice per second
            # At 30fps, 15 frames = 0.5 seconds
            phase = (self.test_frame_count % 15) / 15.0 * 2 * np.pi
            intensity = 180 + 75 * np.sin(phase)
            
            # Yellow color in BGR
            color = (0, int(intensity * 0.9), int(intensity))
            size = np.random.randint(12, 18)
            
            # Draw LED
            cv2.circle(frame, (x, y), size, color, -1)
            # Add slight glow
            cv2.circle(frame, (x, y), size + 5, color, 2)
        
        return frame
    
    def process_frame_for_gui(self, frame):
        """Apply visualization processing"""
        global mode, detection_engine
        
        try:
            try:
                binary_threshold = self.get_live_parameter('BINARY_THRESHOLD_MODE1')
            except Exception:
                binary_threshold = 200
            
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            _, binary_frame = cv2.threshold(gray, binary_threshold, 255, cv2.THRESH_BINARY)
            
            try:
                if mode == 1:
                    processed_frame, detection_info = detection_engine.mode1_frequency_detection_safe(frame, binary_frame, self)
                    
                    if detection_info and hasattr(self, 'last_status_update'):
                        current_time = time.time()
                        if current_time - getattr(self, 'last_status_update', 0) > 1.0:
                            target_cells = sum(1 for info in detection_info if info['status'] == 'TARGET')
                            freq_cells = sum(1 for info in detection_info if info['status'] == 'FREQ')
                            total_cells = len(detection_info)
                            
                            self._safe_gui_update('distance', f"{target_cells} target, {freq_cells} freq")
                            self._safe_gui_update('alignment', f"FFT Analysis")
                            self.last_status_update = current_time
                
                elif mode == 2:
                    processed_frame, detection_info, contours = detection_engine.mode2_flicker_analysis_safe(frame, self)
                    
                    if hasattr(self, 'last_status_update'):
                        current_time = time.time()
                        if current_time - getattr(self, 'last_status_update', 0) > 1.0:
                            flicker_cells = sum(1 for info in detection_info if info.get('status') == 'FLICKER')
                            weak_cells = sum(1 for info in detection_info if info.get('status') == 'WEAK')
                            
                            self._safe_gui_update('distance', f"{flicker_cells} flickering")
                            self._safe_gui_update('alignment', f"{weak_cells} weak")
                            self.last_status_update = current_time
                
                elif mode == 3:
                    processed_frame, detection_info = detection_engine.mode3_object_tracking_safe(frame, binary_frame, self)
                    
                    if detection_info and hasattr(self, 'last_status_update'):
                        current_time = time.time()
                        if current_time - getattr(self, 'last_status_update', 0) > 1.0:
                            objects_found = detection_info.get('objects_found', 0)
                            largest_area = detection_info.get('largest_area', 0)
                            
                            self._safe_gui_update('distance', f"{objects_found} objects")
                            self._safe_gui_update('alignment', f"Max: {largest_area}px")
                            self.last_status_update = current_time
                else:
                    processed_frame = frame.copy()
                    cv2.putText(processed_frame, f"Unknown Mode {mode}", (10, 30), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            except Exception as detection_error:
                print(f"⚠️ Visualization error: {detection_error}")
                processed_frame = frame.copy()
                cv2.putText(processed_frame, f"Visualization Error", (10, 30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
            
            return processed_frame
        except Exception as e:
            print(f"Frame processing error: {e}")
            try:
                error_frame = frame.copy()
                cv2.putText(error_frame, "Processing Error", (10, 30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
                return error_frame
            except Exception:
                return np.zeros((480, 640, 3), dtype=np.uint8)
    
    def update_display(self):
        """Update video display"""
        if self.is_running:
            try:
                while not self.frame_queue.empty():
                    try:
                        frame = self.frame_queue.get_nowait()
                        if frame is not None:
                            self.display_frame(frame)
                    except queue.Empty:
                        break
                    except Exception as e:
                        logging.warning(f"Frame queue error: {e}")
                        break
                
                if self.frame_queue.qsize() > 5:
                    while not self.frame_queue.empty():
                        try:
                            self.frame_queue.get_nowait()
                        except queue.Empty:
                            break
            except Exception as e:
                logging.warning(f"Display update error: {e}")
            
            try:
                self.master.after(50, self.update_display)
            except Exception as e:
                logging.error(f"Display schedule error: {e}")
                if self.is_running:
                    try:
                        self.master.after(100, self.update_display)
                    except Exception:
                        pass
    
    def display_frame(self, frame):
        """Display frame"""
        if frame is None:
            return
        
        try:
            try:
                canvas_width = self.video_canvas.winfo_width()
                canvas_height = self.video_canvas.winfo_height()
            except Exception:
                canvas_width, canvas_height = 576, 432
            
            if canvas_width <= 1 or canvas_height <= 1:
                return
            
            height, width = frame.shape[:2]
            scale = min(canvas_width/width, canvas_height/height, 0.8)
            new_width = max(1, int(width * scale))
            new_height = max(1, int(height * scale))
            
            try:
                if len(frame.shape) == 3 and frame.shape[2] == 3:
                    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                else:
                    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_GRAY2RGB)
            except Exception:
                frame_rgb = np.zeros((height, width, 3), dtype=np.uint8)
            
            try:
                pil_image = Image.fromarray(frame_rgb)
                pil_image = pil_image.resize((new_width, new_height), Image.Resampling.NEAREST)
                photo = ImageTk.PhotoImage(pil_image)
            except Exception as e:
                logging.warning(f"Image conversion error: {e}")
                return
            
            try:
                self.video_canvas.delete("all")
                x = max(0, (canvas_width - new_width) // 2)
                y = max(0, (canvas_height - new_height) // 2)
                self.video_canvas.create_image(x, y, anchor=tk.NW, image=photo)
                self.video_canvas.image = photo
            except Exception as e:
                logging.warning(f"Canvas update error: {e}")
        except Exception as e:
            logging.warning(f"Display frame error: {e}")

def launch_gui():
    """Launch the GUI"""
    root = tk.Tk()
    app = LEDDetectionGUI(root)
    
    def on_closing():
        app.stop_detection()
        root.destroy()
    
    root.protocol("WM_DELETE_WINDOW", on_closing)
    
    root.update_idletasks()
    x = (root.winfo_screenwidth() // 2) - (1260 // 2)
    y = (root.winfo_screenheight() // 2) - (810 // 2)
    root.geometry(f"1260x810+{x}+{y}")
    
    return root, app

# ===== INITIALIZE CAMERA AND SETUP =====

# Initialize camera service variable (but don't connect yet)
camera_service = None
globals()['reucam'] = None
globals()['is_recording'] = False

print("\n✅ Complete GUI system initialized!")
print("🎨 Features:")
print("   • Mode buttons at top of GUI")
print("   • Color legend panel updates per mode")
print("   • Parameter controls with +/- buttons")
print("   • Real-time video display")
print("   • RTSP camera integration")

print("\n📋 Camera Configuration:")
print(f"   RTSP URL: {RTSP_URL}")
print(f"   RTSP Enabled: {USE_RTSP}")
print(f"   Camera will connect when detection starts")

print("\n🚀 To launch GUI: root, gui_app = launch_gui(); root.mainloop()")
print("💡 Camera connects automatically when you click 'Start Detection'")

✅ Essential variables initialized

✅ Complete GUI system initialized!
🎨 Features:
   • Mode buttons at top of GUI
   • Color legend panel updates per mode
   • Parameter controls with +/- buttons
   • Real-time video display
   • RTSP camera integration

📋 Camera Configuration:
   RTSP URL: rtsp://169.254.104.28:8554/0/unicast
   RTSP Enabled: True
   Camera will connect when detection starts

🚀 To launch GUI: root, gui_app = launch_gui(); root.mainloop()
💡 Camera connects automatically when you click 'Start Detection'


In [ ]:
# ^ ===== SIMPLIFIED VISUALIZATION ALGORITHMS =====
# Visual processing only - no complex detection logic

from scipy import ndimage
from scipy.signal import find_peaks

class SimpleVisualizationEngine:
    """Simplified visualization engine for parameter tuning"""
    
    def __init__(self):
        self.frame_count = 0
        self.init_simple_buffers()
        
    def init_simple_buffers(self):
        """Initialize simple visualization buffers"""
        # Simple counters for visualization
        self.frame_count = 0
        self.HE = 0  # Grid height
        self.WI = 0  # Grid width
        
        # Mode 1: FFT frequency detection buffers (per grid cell)
        self.fourier_buff = {}  # Dictionary: (grid_x, grid_y) -> deque of intensity values
        self.freq_counter = 0
        
        # Mode 2: Flicker detection buffers (intensity snapshots per grid cell)
        self.flicker_buff = {}  # Dictionary: (grid_x, grid_y) -> list of [timestamp, mean_intensity]
        self.flicker_stable_count = 0
        
        # Mode 3: Simple tracking
        self.previous_mask = None
        self.tracker = None
        self.bbox_mode3 = None
        self.tracking_counter = 0
        self.tracking_failures = 0
        
        print("🔧 Mode 1: FFT frequency buffers initialized")
        print("🔧 Mode 2: Flicker intensity buffers initialized")
    
    def get_fft_frequency(self, signal, fps=30.0):
        """Calculate dominant frequency using robust FFT analysis"""
        try:
            # Input validation with safety checks
            if signal is None or len(signal) < 10:
                return 0.0
            
            # Convert to numpy array if needed and validate
            signal = np.asarray(signal, dtype=np.float32)
            if signal.size == 0 or np.all(signal == 0):
                return 0.0
            
            # Remove invalid values (NaN, inf)
            signal = signal[np.isfinite(signal)]
            if len(signal) < 10:
                return 0.0
            
            # Remove DC component safely
            try:
                signal_mean = np.mean(signal)
                if np.isfinite(signal_mean):
                    signal_normalized = signal - signal_mean
                else:
                    signal_normalized = signal
            except Exception:
                signal_normalized = signal
            
            # Check if signal has any variation
            if np.std(signal_normalized) < 1e-6:
                return 0.0
            
            # Apply FFT with error handling
            try:
                fft_result = np.fft.rfft(signal_normalized)
                fft_vals = np.abs(fft_result)[1:]  # Skip DC component
                
                if len(fft_vals) == 0:
                    return 0.0
                
                # Create frequency axis with bounds checking
                max_freq = fps / 2.0
                if max_freq <= 0:
                    max_freq = 15.0  # Fallback
                    
                f_axis = np.linspace(0, max_freq, len(fft_vals) + 1)[1:]
                
                if len(f_axis) != len(fft_vals):
                    return 0.0
                    
            except Exception as fft_error:
                print(f"FFT computation error: {fft_error}")
                return 0.0
            
            # Find peaks safely
            try:
                # Calculate threshold for peak detection
                fft_max = np.max(fft_vals)
                fft_median = np.median(fft_vals)
                
                if fft_max <= 0 or not np.isfinite(fft_max):
                    return 0.0
                
                # Use adaptive threshold
                threshold = max(fft_max * 0.3, fft_median * 4)
                
                peaks, _ = find_peaks(fft_vals, height=threshold)
                
                # Return dominant frequency if significant peak found
                if len(peaks) > 0:
                    # Get the strongest peak
                    peak_idx = peaks[np.argmax(fft_vals[peaks])]
                    
                    # Validate peak index
                    if 0 <= peak_idx < len(f_axis):
                        detected_freq = f_axis[peak_idx]
                        
                        # Sanity check on frequency range
                        if 0.1 <= detected_freq <= 20.0:  # Reasonable frequency range
                            return float(detected_freq)
                
                return 0.0
                
            except Exception as peak_error:
                print(f"Peak detection error: {peak_error}")
                return 0.0
            
        except Exception as e:
            print(f"FFT Analysis Error: {e}")
            return 0.0
    
    def mode1_frequency_detection_safe(self, frame, binary_frame, gui_ref):
        """Mode 1: FFT Frequency Detection with readable frequency display on each box"""
        try:
            from collections import deque
            h, w = frame.shape[:2]
            
            # Get parameters from GUI
            try:
                current_box_size = max(10, min(gui_ref.box_size_var.get(), min(h//2, w//2)))
                calc_length = max(20, min(gui_ref.calc_length_var.get(), 300))
            except Exception:
                current_box_size = 100
                calc_length = 60
            
            # Calculate grid dimensions
            grid_h = max(1, h // current_box_size)
            grid_w = max(1, w // current_box_size)
            
            vis_frame = frame.copy()
            detection_info = []
            
            # Initialize buffers if needed
            if not isinstance(self.fourier_buff, dict):
                self.fourier_buff = {}
            
            # Process each grid cell for FFT frequency detection
            for gh in range(grid_h):
                for gw in range(grid_w):
                    try:
                        # Calculate grid cell bounds
                        y1 = gh * current_box_size
                        x1 = gw * current_box_size
                        y2 = min((gh + 1) * current_box_size, h)
                        x2 = min((gw + 1) * current_box_size, w)
                        
                        # Extract ROI and calculate mean intensity
                        roi_binary = binary_frame[y1:y2, x1:x2]
                        mean_intensity = np.mean(roi_binary)
                        
                        # Initialize buffer for this grid cell if needed
                        grid_key = (gw, gh)
                        if grid_key not in self.fourier_buff:
                            self.fourier_buff[grid_key] = deque(maxlen=calc_length)
                        
                        # Add intensity to buffer
                        self.fourier_buff[grid_key].append(mean_intensity)
                        
                        # Calculate frequency if buffer is full
                        detected_freq = 0.0
                        freq_confidence = 0.0
                        if len(self.fourier_buff[grid_key]) >= calc_length:
                            signal = list(self.fourier_buff[grid_key])
                            detected_freq = self.get_fft_frequency(signal, fps=30.0)
                            freq_confidence = mean_intensity / 255.0
                        
                        # Color coding based on detected frequency
                        if detected_freq >= 1.5 and detected_freq <= 2.5:  # Near 2Hz target
                            color = (0, 255, 0)  # Green: Target frequency detected
                            status = "TARGET"
                        elif detected_freq > 0.5:
                            color = (0, 255, 255)  # Yellow: Some frequency detected
                            status = "FREQ"
                        elif mean_intensity > 100:
                            color = (255, 150, 0)  # Blue: Bright but no frequency
                            status = "BRIGHT"
                        else:
                            color = (100, 100, 100)  # Gray: Low intensity
                            status = "LOW"
                        
                        # Draw grid cell
                        cv2.rectangle(vis_frame, (x1, y1), (x2, y2), color, 2)
                        
                        # Show frequency with LARGER, READABLE text
                        if (x2-x1) > 50 and (y2-y1) > 40:
                            if detected_freq > 0.1:
                                # Show frequency in Hz with larger font
                                freq_text = f"{detected_freq:.1f}Hz"
                                cv2.putText(vis_frame, freq_text, 
                                           (x1+5, y1+25), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
                            else:
                                # Show "Analyzing..." while building buffer
                                cv2.putText(vis_frame, "Wait...", 
                                           (x1+5, y1+25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
                            
                            # Show intensity on second line
                            cv2.putText(vis_frame, f"{int(mean_intensity)}", 
                                       (x1+5, y1+45), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
                        
                        detection_info.append({
                            'grid': (gw, gh),
                            'frequency': detected_freq,
                            'intensity': mean_intensity,
                            'status': status
                        })
                        
                    except Exception as e:
                        print(f"Grid cell error: {e}")
                        continue
            
            # Add summary with frequency info
            target_cells = sum(1 for info in detection_info if info['status'] == 'TARGET')
            freq_cells = sum(1 for info in detection_info if info['status'] == 'FREQ')
            cv2.putText(vis_frame, f"Mode 1: {target_cells} at 2Hz, {freq_cells} other frequencies", 
                       (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
            cv2.putText(vis_frame, f"Grid: {grid_w}x{grid_h}, Buffer: {calc_length} frames", 
                       (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
            
            return vis_frame, detection_info
            
        except Exception as e:
            print(f"Mode 1 FFT Error: {e}")
            return self._safe_error_frame(frame, "Mode 1 FFT Error"), []
    
    def mode2_flicker_analysis_safe(self, frame, gui_ref):
        """Mode 2: Yellow HSV filtering with grid-based flicker detection using intensity subtraction"""
        try:
            import time
            h, w = frame.shape[:2]
            
            # Get parameters from GUI
            try:
                current_box_size = max(10, min(gui_ref.box_size_var.get(), min(h//2, w//2)))
                hsv_lower = gui_ref.get_live_parameter('HSV_LOWER_BOUND')
                hsv_upper = gui_ref.get_live_parameter('HSV_UPPER_BOUND')
            except Exception:
                current_box_size = 100
                # Yellow color range in HSV
                hsv_lower = np.array([20, 100, 100])  # Yellow lower bound
                hsv_upper = np.array([30, 255, 255])  # Yellow upper bound
            
            # Apply yellow HSV filtering
            hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
            mask = cv2.inRange(hsv, hsv_lower, hsv_upper)
            
            # Create visualization with yellow overlay
            colored_mask = cv2.applyColorMap(mask, cv2.COLORMAP_HOT)
            vis_frame = cv2.addWeighted(frame, 0.7, colored_mask, 0.3, 0)
            
            # Calculate grid dimensions
            grid_h = max(1, h // current_box_size)
            grid_w = max(1, w // current_box_size)
            
            # Initialize flicker buffers if needed
            if not isinstance(self.flicker_buff, dict):
                self.flicker_buff = {}
            
            current_time = time.time()
            detection_info = []
            
            # Process each grid cell for flicker detection
            for gh in range(grid_h):
                for gw in range(grid_w):
                    try:
                        # Calculate grid cell bounds
                        y1 = gh * current_box_size
                        x1 = gw * current_box_size
                        y2 = min((gh + 1) * current_box_size, h)
                        x2 = min((gw + 1) * current_box_size, w)
                        
                        # Extract ROI from HSV mask and calculate mean intensity
                        roi_mask = mask[y1:y2, x1:x2]
                        mean_intensity = np.mean(roi_mask)
                        
                        # Initialize buffer for this grid cell if needed
                        grid_key = (gw, gh)
                        if grid_key not in self.flicker_buff:
                            self.flicker_buff[grid_key] = []
                        
                        # Add intensity snapshot with timestamp (every 0.25 sec for 2Hz LEDs)
                        self.flicker_buff[grid_key].append([current_time, mean_intensity])
                        
                        # Keep only last 2 seconds of data
                        self.flicker_buff[grid_key] = [
                            [t, i] for t, i in self.flicker_buff[grid_key] 
                            if current_time - t <= 2.0
                        ]
                        
                        # Calculate flicker by comparing intensities
                        flicker_detected = False
                        intensity_diff = 0
                        
                        if len(self.flicker_buff[grid_key]) >= 2:
                            # Get intensity values from last 0.5 sec (2 calculations for 2Hz)
                            recent = [i for t, i in self.flicker_buff[grid_key] if current_time - t <= 0.5]
                            
                            if len(recent) >= 2:
                                # Calculate intensity difference (flicker detection)
                                intensity_diff = abs(max(recent) - min(recent))
                                
                                # Flicker detected if difference is significant
                                if intensity_diff > 30:  # Threshold for flicker
                                    flicker_detected = True
                        
                        # Color coding based on flicker detection
                        if flicker_detected and intensity_diff > 100:
                            color = (0, 0, 255)  # Red: Strong flicker detected (LED!)
                            status = "FLICKER"
                        elif flicker_detected:
                            color = (0, 255, 255)  # Yellow: Weak flicker
                            status = "WEAK"
                        elif mean_intensity > 50:
                            color = (255, 150, 0)  # Blue: Yellow detected but no flicker
                            status = "YELLOW"
                        else:
                            color = (100, 100, 100)  # Gray: Nothing
                            status = "NONE"
                        
                        # Draw grid cell
                        cv2.rectangle(vis_frame, (x1, y1), (x2, y2), color, 2)
                        
                        # Show flicker info with LARGER text
                        if (x2-x1) > 50 and (y2-y1) > 40:
                            if flicker_detected:
                                # Show "FLICKER" text
                                cv2.putText(vis_frame, "FLICKER", 
                                           (x1+5, y1+25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
                                # Show intensity difference
                                cv2.putText(vis_frame, f"Δ{int(intensity_diff)}", 
                                           (x1+5, y1+45), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
                            else:
                                # Show current intensity
                                cv2.putText(vis_frame, f"{int(mean_intensity)}", 
                                           (x1+5, y1+25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
                        
                        detection_info.append({
                            'grid': (gw, gh),
                            'intensity': mean_intensity,
                            'intensity_diff': intensity_diff,
                            'flicker': flicker_detected,
                            'status': status
                        })
                        
                    except Exception as e:
                        print(f"Grid cell error: {e}")
                        continue
            
            # Add summary with flicker info
            flicker_cells = sum(1 for info in detection_info if info['status'] == 'FLICKER')
            weak_cells = sum(1 for info in detection_info if info['status'] == 'WEAK')
            cv2.putText(vis_frame, f"Mode 2: {flicker_cells} flickering LEDs, {weak_cells} weak flickers", 
                       (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
            cv2.putText(vis_frame, f"Yellow HSV filter, Grid: {grid_w}x{grid_h}", 
                       (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
            
            return vis_frame, detection_info, []  # Return empty contours for compatibility
            
        except Exception as e:
            print(f"Mode 2 Flicker Error: {e}")
            return self._safe_error_frame(frame, "Mode 2 Flicker Error"), [], []
    
    def mode3_object_tracking_safe(self, frame, binary_frame, gui_ref):
        """Mode 3: Simple Object Info Display (No Tracking Logic)"""
        try:
            vis_frame = frame.copy()
            
            # Simple object analysis without tracking
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
            contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            # Show largest objects for reference (no actual tracking)
            sorted_contours = []
            if len(contours) > 0:
                # Sort by area and show top 3
                sorted_contours = sorted(contours, key=cv2.contourArea, reverse=True)[:3]
                
                for i, contour in enumerate(sorted_contours):
                    area = cv2.contourArea(contour)
                    if area > 100:
                        x, y, w, h = cv2.boundingRect(contour)
                        
                        # Different colors for largest objects
                        colors = [(0, 255, 0), (0, 255, 255), (255, 0, 255)]
                        color = colors[i] if i < len(colors) else (128, 128, 128)
                        
                        # Draw bounding box with enhanced visibility
                        cv2.rectangle(vis_frame, (x, y), (x+w, y+h), color, 2)
                        cv2.putText(vis_frame, f"Obj{i+1}: {int(area)}px", 
                                   (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
                        
                        # Show center point
                        center_x = x + w // 2
                        center_y = y + h // 2
                        cv2.circle(vis_frame, (center_x, center_y), 4, color, -1)
            
            # Add summary info (no legend on video)
            cv2.putText(vis_frame, f"Mode 3: {len(sorted_contours)} objects detected", 
                       (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
            cv2.putText(vis_frame, "Visual analysis only - no tracking", 
                       (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
            
            detection_info = {
                'objects_found': len(sorted_contours),
                'largest_area': int(cv2.contourArea(sorted_contours[0])) if sorted_contours else 0,
                'status': 'VISUAL_ONLY'
            }
            
            return vis_frame, detection_info
            
        except Exception as e:
            print(f"Mode 3 Visualization Error: {e}")
            return self._safe_error_frame(frame, "Mode 3 Viz Error"), {}
    
    def _safe_error_frame(self, frame, message):
        """Create safe error frame without crashing"""
        try:
            error_frame = frame.copy()
            cv2.putText(error_frame, message, (10, 30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
            return error_frame
        except Exception:
            return np.zeros((480, 640, 3), dtype=np.uint8)

# Create global visualization engine
detection_engine = SimpleVisualizationEngine()

print("🎨 Enhanced LED detection algorithms implemented!")
print("✅ Mode 1: FFT Frequency Detection")
print("   • Detects frequencies using FFT analysis")
print("   • Shows frequency (Hz) on each grid box with LARGE text")
print("   • Green = 2Hz target, Yellow = other frequencies")
print("✅ Mode 2: Yellow HSV + Flicker Detection")
print("   • Filters yellow color with HSV")
print("   • Grid-based flicker detection using intensity subtraction")
print("   • Calculates intensity every 0.25 sec, compares over 0.5 sec")
print("   • Red = Strong flicker (LED detected!)")
print("✅ Mode 3: Object visualization (unchanged)")
print("\n🚀 Ready to launch GUI!")

🔧 Mode 1: FFT frequency buffers initialized
🔧 Mode 2: Flicker intensity buffers initialized
🎨 Enhanced LED detection algorithms implemented!
✅ Mode 1: FFT Frequency Detection
   • Detects frequencies using FFT analysis
   • Shows frequency (Hz) on each grid box with LARGE text
   • Green = 2Hz target, Yellow = other frequencies
✅ Mode 2: Yellow HSV + Flicker Detection
   • Filters yellow color with HSV
   • Grid-based flicker detection using intensity subtraction
   • Calculates intensity every 0.25 sec, compares over 0.5 sec
   • Red = Strong flicker (LED detected!)
✅ Mode 3: Object visualization (unchanged)

🚀 Ready to launch GUI!


In [ ]:
# ^ ===== LAUNCH WORKING GUI =====
# Start the fully functional LED Detection GUI

print("🚀 Launching LED Detection GUI...")

# Close any existing GUI first
try:
    if 'gui_root' in globals() and gui_root:
        gui_root.quit()
        gui_root.destroy()
    if 'gui_application' in globals() and gui_application:
        gui_application.stop_detection()
except:
    pass

try:
    # Create the GUI
    gui_root, gui_application = launch_gui()
    
    print("✅ GUI Created Successfully!")
    print("\n📋 Usage Instructions:")
    print("1. The GUI window should be visible on your screen")
    print("2. Use the parameter tabs on the right to adjust detection settings")
    print("3. Click 'Start Detection' to begin LED detection")
    print("4. Camera should connect automatically (RTSP is ready)")
    print("5. Use 'Force Mode 1' button to reset detection mode")
    print("6. Close the window when finished")
    
    print(f"\n📷 Camera Settings:")
    print(f"• RTSP: {'Enabled' if USE_RTSP else 'Disabled'}")
    print(f"• ReuCamera: {'Available' if HAS_REUCAMERA else 'Not Available'}")
    
    print("\n🔄 Starting GUI mainloop...")
    gui_root.mainloop()
    
    print("✅ GUI closed successfully!")
    
except Exception as e:
    print(f"❌ GUI Error: {e}")
    print(f"Error Type: {type(e).__name__}")
    if "logging" in str(e):
        print("💡 Tip: Missing import resolved in GUI application cell")
    elif "cv2" in str(e):
        print("💡 Tip: OpenCV import issue - check if opencv-python is installed")
    else:
        print("💡 Tip: Run cells 2 and 3 first if GUI class not found")

🚀 Launching LED Detection GUI...
🔧 GUI initialized
✅ GUI Created Successfully!

📋 Usage Instructions:
1. The GUI window should be visible on your screen
2. Use the parameter tabs on the right to adjust detection settings
3. Click 'Start Detection' to begin LED detection
4. Camera should connect automatically (RTSP is ready)
5. Use 'Force Mode 1' button to reset detection mode
6. Close the window when finished

📷 Camera Settings:
• RTSP: Enabled
• ReuCamera: Not Available

🔄 Starting GUI mainloop...
🔧 GUI initialized
✅ GUI Created Successfully!

📋 Usage Instructions:
1. The GUI window should be visible on your screen
2. Use the parameter tabs on the right to adjust detection settings
3. Click 'Start Detection' to begin LED detection
4. Camera should connect automatically (RTSP is ready)
5. Use 'Force Mode 1' button to reset detection mode
6. Close the window when finished

📷 Camera Settings:
• RTSP: Enabled
• ReuCamera: Not Available

🔄 Starting GUI mainloop...
🔌 Connecting to RTSP came

2025-11-19 10:20:52,200 - WARNING - GUI update failed for distance: main thread is not in main loop
2025-11-19 10:20:53,286 - WARNING - GUI update failed for alignment: main thread is not in main loop
2025-11-19 10:20:53,286 - WARNING - GUI update failed for alignment: main thread is not in main loop
2025-11-19 10:20:54,338 - WARNING - GUI update failed for fps: main thread is not in main loop
2025-11-19 10:20:54,338 - WARNING - GUI update failed for fps: main thread is not in main loop
2025-11-19 10:20:55,405 - WARNING - GUI update failed for mode: main thread is not in main loop
2025-11-19 10:20:55,405 - WARNING - GUI update failed for mode: main thread is not in main loop
2025-11-19 10:20:56,466 - WARNING - GUI update failed for camera: main thread is not in main loop
2025-11-19 10:20:56,466 - WARNING - GUI update failed for camera: main thread is not in main loop
